In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor


In [2]:
data = pd.read_csv('../../datasets/winequality-white.csv')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [6]:
y = data.quality
X = data.drop(['quality'], axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [8]:
scaler = StandardScaler()

In [9]:
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform((X_test))

In [17]:
linreg = LinearRegression()
clf = linreg.fit(X_train_scaled, y_train)

In [14]:
print("Mean squared error (train): %.3f" % mean_squared_error(linreg.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(linreg.predict(X_holdout_scaled), y_test))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


In [24]:
clf.coef_

array([ 9.78219223e-02, -1.92259947e-01, -1.83224449e-04,  5.38164096e-01,
        8.12724353e-03,  4.21804406e-02,  1.43040227e-02, -6.65720472e-01,
        1.50036006e-01,  6.20533605e-02,  1.29533447e-01])

In [27]:
linreg_coef = pd.DataFrame({'importance': clf.coef_}, index=X.columns)

In [30]:
linreg_coef.sort_values(by='importance', ascending=False)

,importance
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


In [31]:
lasso1 = Lasso(random_state=17, alpha=.01)
lasso1.fit(X_train_scaled, y_train)


Lasso(alpha=0.01, random_state=17)

In [33]:
lasso1_coef = pd.DataFrame({'importance': lasso1.coef_}, index=X.columns)

In [35]:
lasso1_coef.sort_values(by='importance', ascending=True)

,importance
density,-0.235492
volatile acidity,-0.188479
chlorides,-0.002747
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
sulphates,0.029722
free sulfur dioxide,0.043088
pH,0.067277
residual sugar,0.256363


In [36]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, alphas=alphas, cv=5)

In [37]:
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [38]:
lasso_cv.alpha_

0.0002833096101839324

In [39]:
lasso_cv_coef = pd.DataFrame({'importance': lasso_cv.coef_}, index=X.columns)

In [42]:
lasso_cv_coef.sort_values(by='importance', ascending=True)

,importance
density,-0.648161
volatile acidity,-0.192049
citric acid,-0.000000
chlorides,0.006933
total sulfur dioxide,0.012969
free sulfur dioxide,0.042698
sulphates,0.060939
fixed acidity,0.093295
alcohol,0.137115
pH,0.146549


In [43]:
print("Mean squared error (train): %.3f" % mean_squared_error(lasso_cv.predict(X_train_scaled), y_train))
print("Mean squared error (test): %.3f" % mean_squared_error(lasso_cv.predict(X_holdout_scaled), y_test))


Mean squared error (train): 0.558
Mean squared error (test): 0.583


In [44]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)


RandomForestRegressor(random_state=17)

In [46]:
forest.score(X_train, y_train)

-1.7838588723293292

In [49]:
print("Mean squared error (train): %.3f" % mean_squared_error(forest.predict(X_train), y_train))
print("Mean squared error (cv): %.3f" % cross_val_score(forest, X_train, y=y_train, scoring='neg_mean_squared_error'))
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_test), y_test))

Mean squared error (train): 0.053


TypeError: only size-1 arrays can be converted to Python scalars

In [50]:
print("Mean squared error (test): %.3f" % mean_squared_error(forest.predict(X_test), y_test))

Mean squared error (test): 0.371


In [56]:
print("Mean squared error (cv): %.3f" % cross_val_score(forest, X_train, y_train))


TypeError: only size-1 arrays can be converted to Python scalars

In [57]:
forest_params = {'max_depth': list(range(10, 25)),
                  'min_samples_leaf': list(range(1, 8)),
                  'max_features': list(range(6,12))}
locally_best_forest = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, return_train_score=True)

In [58]:
locally_best_forest.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
             return_train_score=True)

In [59]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4938417994321389)

In [61]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_,
                                                        X_train_scaled, y_train,
                                                        scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_test,
                                                             locally_best_forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.398
Mean squared error (test): 2.078


In [62]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_,
                             columns=['coef'], index=data.columns[:-1])
rf_importance.sort_values(by='coef', ascending=False)

,coef
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813


In [ ]:
#The depency of wine quality on other features in hand is,
# presumable,
# non-linear.
# So Random Forest works better in this task.